In [21]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten,BatchNormalization,Conv2D,MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import cv2


In [ ]:
dataset_path=r"C:\Users\Ashok\puthon\Great Learning\Transfer Learning\17flowers\train"

In [2]:
#test a image shape
test_image=cv2.imread(r"C:\Users\Ashok\puthon\Great Learning\Transfer Learning\17flowers\train\1\image_0082.jpg",1)
cv2.imshow('test',test_image)
test_image.shape

(666, 500, 3)

# Organize Data - Split Train Test Validation

In [4]:
train_folder=r"C:\Users\Ashok\puthon\Great Learning\Transfer Learning\17flowers\train"
test_folder=r"C:\Users\Ashok\puthon\Great Learning\Transfer Learning\17flowers\test"
validation_folder=r"C:\Users\Ashok\puthon\Great Learning\Transfer Learning\17flowers\validation"

In [ ]:


for i,(dirpath,dirnames,filenames) in enumerate ((os.walk(dataset_path))):
    if dirpath is not dataset_path:
        print(dirpath)
        
        #Create folders in test path under each class
        class_folder=dirpath.split("\\")[-1]
        test_path=os.path.join(test_folder,class_folder)
        os.mkdir(test_path)
        #create folders in validation path under each class
        validation_path=os.path.join(validation_folder,class_folder)
        os.mkdir(validation_path)
        
        train_path=os.path.join(train_folder,class_folder)
        
        n_files=int(len(os.listdir(train_path))*20/100)
        
        for c in random.sample(os.listdir(train_path),n_files):
            shutil.move(os.path.join(dirpath,c),test_path)
            
        for c in random.sample(os.listdir(train_path),n_files):
            shutil.move(os.path.join(dirpath,c),validation_path)

In [6]:
classes=os.listdir(train_folder)


In [22]:
train_batches=ImageDataGenerator(preprocessing_function=None)\
.flow_from_directory(directory=train_folder,target_size=(224,224),classes=classes,batch_size=10)

validation_batches=ImageDataGenerator(preprocessing_function=None)\
.flow_from_directory(directory=validation_folder,target_size=(224,224),classes=classes,batch_size=10)

test_batches=ImageDataGenerator(preprocessing_function=None)\
.flow_from_directory(directory=test_folder,target_size=(224,224),classes=classes,batch_size=10,shuffle=False)


Found 827 images belonging to 17 classes.
Found 274 images belonging to 17 classes.
Found 274 images belonging to 17 classes.


In [ ]:
imgs,labels=next(train_batches)

# Build CNN

In [23]:
cnn_model=Sequential([
    Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same',input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2,2),strides=2),
    Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
    MaxPool2D(pool_size=(2,2),strides=2),
    Flatten(),
    Dense(units=17,activation='softmax')
])

In [ ]:
cnn_model.summary()

In [24]:
cnn_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [25]:
cnn_model.fit(x=train_batches,validation_data=validation_batches,epochs=10)

Epoch 1/10
83/83 [==============================] - 23s 266ms/step - loss: 266.7020 - accuracy: 0.0821 - val_loss: 2.7379 - val_accuracy: 0.1350
Epoch 2/10
83/83 [==============================] - 22s 262ms/step - loss: 1.7143 - accuracy: 0.5882 - val_loss: 3.2319 - val_accuracy: 0.1971
Epoch 3/10
83/83 [==============================] - 22s 264ms/step - loss: 0.4566 - accuracy: 0.8874 - val_loss: 3.8446 - val_accuracy: 0.2190
Epoch 4/10
83/83 [==============================] - 22s 266ms/step - loss: 0.1177 - accuracy: 0.9730 - val_loss: 4.9285 - val_accuracy: 0.2226
Epoch 5/10
83/83 [==============================] - 22s 267ms/step - loss: 0.0640 - accuracy: 0.9882 - val_loss: 4.6622 - val_accuracy: 0.2117
Epoch 6/10
83/83 [==============================] - 22s 267ms/step - loss: 0.0317 - accuracy: 0.9950 - val_loss: 4.8200 - val_accuracy: 0.2226
Epoch 7/10
83/83 [==============================] - 23s 279ms/step - loss: 0.0453 - accuracy: 0.9873 - val_loss: 5.0789 - val_accuracy: 0.24

## Create a Transfer Learning Model

In [8]:
# VGG Net
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16

image_input=Input(shape=(224,224,3))
model=VGG16(input_tensor=image_input , weights='imagenet',include_top=False)




In [9]:
#make the layers in the VGG net non trainable
for layer in model.layers:
    layer.trainable=False
    
#vgg.include_top=True
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPool2D,Activation,Dropout,Flatten
from tensorflow.keras.models import Model
import random as rn


In [17]:
block5_pool =model.get_layer('block5_pool').output
block6_conv1=Conv2D(filters=32,kernel_size=(3,3),strides=(2,2),padding='valid',data_format='channels_last',
            activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='block6_conv1')(block5_pool)
block6_pool=MaxPool2D(pool_size=(2,2),strides=(2,2),padding='valid',data_format='channels_last',
               name='block6_pool')(block6_conv1)

#Flatten
fatten=Flatten(data_format='channels_last',name='Flatten')(block6_pool)

#FC layer
FC1=Dense(units=32,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal
         (seed=32),name='FC1')(fatten)

out=Dense(units=17,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal
         (seed=32),name='out')(FC1)

In [18]:
custom_model=Model(image_input,out)
custom_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [19]:
#compile model
custom_model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
custom_model.fit(x=train_batches,validation_data=validation_batches,epochs=20)

Epoch 1/20
83/83 [==============================] - 139s 2s/step - loss: 8.6077 - accuracy: 0.1833 - val_loss: 2.2641 - val_accuracy: 0.4015
Epoch 2/20
83/83 [==============================] - 143s 2s/step - loss: 1.4898 - accuracy: 0.5818 - val_loss: 1.9698 - val_accuracy: 0.5146
Epoch 3/20
83/83 [==============================] - 142s 2s/step - loss: 0.6302 - accuracy: 0.8221 - val_loss: 1.9794 - val_accuracy: 0.6058
Epoch 4/20
83/83 [==============================] - 144s 2s/step - loss: 0.3721 - accuracy: 0.8919 - val_loss: 1.7200 - val_accuracy: 0.6350
Epoch 5/20
83/83 [==============================] - 142s 2s/step - loss: 0.2210 - accuracy: 0.9398 - val_loss: 1.6952 - val_accuracy: 0.6496
Epoch 6/20
83/83 [==============================] - 145s 2s/step - loss: 0.1483 - accuracy: 0.9464 - val_loss: 2.0021 - val_accuracy: 0.6387
Epoch 7/20
83/83 [==============================] - 143s 2s/step - loss: 0.1044 - accuracy: 0.9693 - val_loss: 1.7251 - val_accuracy: 0.7007
Epoch 8/20
83

# Observstions

- Since Transfer Learning uses VGG16 which was balically trained on a large amount of data set, we can see a substantial increase in accuracy
- VGG16 also provided a better validation accuracy